In [11]:
# 패션 MNIST - 훈련세트, 테스트 세트
from tensorflow import keras

(train_input, train_target) , (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [12]:
# 모델 생성 함수추가해서 층 추가가 가능하게 만들꺼
def model_fn(a_layer = None):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28,28))) # 입력층
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(100, activation='relu')) # 음수일땐 0으로 하고 양수일때는 통과시킴
    if a_layer:
        model.add(a_layer)
    
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [13]:
model = model_fn(keras.layers.Dropout(0.3)) 
model.summary()
# 비율은 찾아가는거 0.1이 될 수도 0.2가 될수도

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)                  │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
train_scaled = train_input / 255
test_scaled = test_input / 255
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대 적합 직전 시점에 모델을 파일로 저장!
checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras", save_best_only=True) 

# 과대 적합 되기 직전 까지만 훈련하고 종료 - 조기종료
# 검증 세트의 정확도가 2회 이상 올라가지 않으면 조기 종료
# restore_best_weights : 가장 최적의 가중치로 복구
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

model.fit(train_scaled, train_target, epochs=20, validation_data=(test_scaled, test_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7380 - loss: 0.7475 - val_accuracy: 0.8447 - val_loss: 0.4445
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8416 - loss: 0.4393 - val_accuracy: 0.8533 - val_loss: 0.4081
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8528 - loss: 0.4025 - val_accuracy: 0.8640 - val_loss: 0.3752
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8622 - loss: 0.3728 - val_accuracy: 0.8659 - val_loss: 0.3753
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8683 - loss: 0.3605 - val_accuracy: 0.8690 - val_loss: 0.3644
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8718 - loss: 0.3441 - val_accuracy: 0.8672 - val_loss: 0.3651
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8774 - loss: 0.3331 - val_accuracy: 0.8748 - val_loss: 0.3470
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8768 - loss: 0.3297 - 

In [15]:
model = keras.models.load_model("best-model.keras")

In [16]:
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.8761 - loss: 0.3336


[0.3413896858692169, 0.8748000264167786]

In [17]:
# 조기 종료 시점
early_stopping_cb.stopped_epoch

10